In [1]:
#  imports
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime
import os

# for nicer display in notebooks
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

In [2]:
# load dataset
csv_path = "Online Sales Data.csv"
df = pd.read_csv(csv_path)

# quick peek
print("Rows, cols:", df.shape)
df.head()


Rows, cols: (240, 9)


,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


In [5]:
#  normalize column names and parse date
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]


expected = ['transaction_id','date','product_category','product_name','units_sold','unit_price','total_revenue','region','payment_method']
missing = [c for c in expected if c not in df.columns]
print("Missing expected columns:", missing)

# parse date
df['date'] = pd.to_datetime(df['date'], errors='coerce')
print("Null dates:", df['date'].isna().sum())

# ensure numeric columns are numeric
for col in ['units_sold','unit_price','total_revenue']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df.info()


Missing expected columns: []
Null dates: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    240 non-null    int64         
 1   date              240 non-null    datetime64[ns]
 2   product_category  240 non-null    object        
 3   product_name      240 non-null    object        
 4   units_sold        240 non-null    int64         
 5   unit_price        240 non-null    float64       
 6   total_revenue     240 non-null    float64       
 7   region            240 non-null    object        
 8   payment_method    240 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 17.0+ KB


In [6]:
#  create SQLite DB file in the notebook folder
db_file = 'online_sales.db'
if os.path.exists(db_file):
    os.remove(db_file)

conn = sqlite3.connect(db_file)
df.to_sql('online_sales', conn, index=False)
print("Wrote", len(df), "rows to SQLite table 'online_sales' in", db_file)


Wrote 240 rows to SQLite table 'online_sales' in online_sales.db


In [7]:
#  EXTRACT(MONTH FROM order_date) for month.
query_a = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    *
FROM online_sales
LIMIT 5;
"""
pd.read_sql_query(query_a, conn)


,year,month,transaction_id,date,product_category,product_name,units_sold,unit_price,total_revenue,region,payment_method
0,2024,01,10001,2024-01-01 00:00:00,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,2024,01,10002,2024-01-02 00:00:00,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,2024,01,10003,2024-01-03 00:00:00,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,2024,01,10004,2024-01-04 00:00:00,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,2024,01,10005,2024-01-05 00:00:00,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal


In [8]:
# GROUP BY year/month
query_b = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    COUNT(*) AS total_transactions
FROM online_sales
GROUP BY year, month
ORDER BY year, month;
"""
pd.read_sql_query(query_b, conn)


,year,month,total_transactions
0,2024,01,31
1,2024,02,29
2,2024,03,31
3,2024,04,30
4,2024,05,31
5,2024,06,30
6,2024,07,31
7,2024,08,27


In [9]:
#  SUM() for revenue.
query_c = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    SUM(total_revenue) AS monthly_revenue
FROM online_sales
GROUP BY year, month
ORDER BY year, month;
"""
pd.read_sql_query(query_c, conn)


,year,month,monthly_revenue
0,2024,01,14548.32
1,2024,02,10803.37
2,2024,03,12849.24
3,2024,04,12451.69
4,2024,05,8455.49
5,2024,06,7384.55
6,2024,07,6797.08
7,2024,08,7278.11


In [10]:
#  COUNT(DISTINCT order_id) for volume
query_d = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    COUNT(DISTINCT transaction_id) AS order_volume
FROM online_sales
GROUP BY year, month
ORDER BY year, month;
"""
pd.read_sql_query(query_d, conn)


,year,month,order_volume
0,2024,01,31
1,2024,02,29
2,2024,03,31
3,2024,04,30
4,2024,05,31
5,2024,06,30
6,2024,07,31
7,2024,08,27


In [11]:
# ORDER BY for sorting
query_e = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    SUM(total_revenue) AS total_revenue,
    COUNT(DISTINCT transaction_id) AS total_orders
FROM online_sales
GROUP BY year, month
ORDER BY year, month;
"""
monthly_summary = pd.read_sql_query(query_e, conn)
monthly_summary


,year,month,total_revenue,total_orders
0,2024,01,14548.32,31
1,2024,02,10803.37,29
2,2024,03,12849.24,31
3,2024,04,12451.69,30
4,2024,05,8455.49,31
5,2024,06,7384.55,30
6,2024,07,6797.08,31
7,2024,08,7278.11,27


In [12]:
# Limit results for specific time periods
year_filter = '2024'  
query_f = f"""
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    SUM(total_revenue) AS total_revenue,
    COUNT(DISTINCT transaction_id) AS total_orders
FROM online_sales
WHERE STRFTIME('%Y', date) = '{year_filter}'
GROUP BY year, month
ORDER BY month;
"""
pd.read_sql_query(query_f, conn)


,year,month,total_revenue,total_orders
0,2024,01,14548.32,31
1,2024,02,10803.37,29
2,2024,03,12849.24,31
3,2024,04,12451.69,30
4,2024,05,8455.49,31
5,2024,06,7384.55,30
6,2024,07,6797.08,31
7,2024,08,7278.11,27


In [13]:
#  top 3 months overall by revenue
query_top3 = """
SELECT
    STRFTIME('%Y', date) AS year,
    STRFTIME('%m', date) AS month,
    SUM(total_revenue) AS total_revenue
FROM online_sales
GROUP BY year, month
ORDER BY total_revenue DESC
LIMIT 3;
"""
pd.read_sql_query(query_top3, conn)


,year,month,total_revenue
0,2024,01,14548.32
1,2024,03,12849.24
2,2024,04,12451.69
